In [20]:
import os
import random
import sqlite3
import sys
import time
from datetime import date, datetime, timedelta
import pandas as pd

import toml
import requests
from tqdm import tqdm


In [28]:
data_headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Host': 'stats.nba.com',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://www.nba.com/',
    'Connection': 'keep-alive'
}


def to_data_frame(data):
    try:
        data_list = data[0]
    except Exception as e:
        print(e)
        return pd.DataFrame(data={})
    return pd.DataFrame(data=data_list.get('rowSet'), columns=data_list.get('headers'))

def get_json_data(url):
    raw_data = requests.get(url, headers=data_headers)
    try:
        json = raw_data.json()
    except Exception as e:
        print(e)
        return {}
    return json.get('resultSets')

In [29]:
url = 'https://stats.nba.com/stats/' \
      'leaguedashteamstats?Conference=&' \
      'DateFrom=10%2F01%2F{2}&DateTo={0}%2F{1}%2F{3}' \
      '&Division=&GameScope=&GameSegment=&LastNGames=0&' \
      'LeagueID=00&Location=&MeasureType=Base&Month=0&' \
      'OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&' \
      'PerMode=PerGame&Period=0&PlayerExperience=&' \
      'PlayerPosition=&PlusMinus=N&Rank=N&' \
      'Season={4}' \
      '&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&' \
      'StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='

year = [2022, 2023]
season = ["2022-23"]

month = [10, 11, 12, 1, 2, 3, 4, 5, 6]


days = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

begin_year_pointer = year[0]
end_year_pointer = year[0]
count = 0

In [30]:
con = sqlite3.connect("C:/Users/mrsam/Documents/Python Test Projects/NBA_dash/Data/teams.sqlite")


for season1 in tqdm(season):
    for month1 in tqdm(month):
        if month1 == 1:
            count += 1
            end_year_pointer = year[count]
        for day1 in tqdm(days):
            if month1 == 10 and day1 < 19:
                continue
            if month1 in [4,6,9,11] and day1 > 30:
                continue
            if month1 == 2 and day1 > 28:
                continue
            if end_year_pointer == datetime.now().year:
                if month1 == datetime.now().month and day1 > datetime.now().day:
                    continue
                if month1 > datetime.now().month:
                    continue
            general_data = get_json_data(url.format(month1, day1, begin_year_pointer, end_year_pointer, season1))
            general_df = to_data_frame(general_data)
            real_date = date(year=end_year_pointer, month=month1, day=day1) + timedelta(days=1)
            general_df['Date'] = str(real_date)

            x = str(real_date).split('-')
            general_df.to_sql(f"teams_{season1}-{str(int(x[1]))}-{str(int(x[2]))}", con, if_exists="replace")
            
            time.sleep(random.randint(1, 3))
    begin_year_pointer = year[count]

con.close()

  0%|          | 0/1 [00:00<?, ?it/s]












100%|██████████| 31/31 [00:29<00:00,  1.05it/s]































100%|██████████| 31/31 [01:22<00:00,  2.65s/it]
































100%|██████████| 31/31 [01:37<00:00,  3.15s/it]
































100%|██████████| 31/31 [01:39<00:00,  3.22s/it]





























100%|██████████| 31/31 [01:19<00:00,  2.55s/it]
































100%|██████████| 31/31 [01:35<00:00,  3.09s/it]































100%|██████████| 31/31 [01:37<00:00,  3.15s/it]
































100%|██████████| 31/31 [01:33<00:00,  3.01s/it]































100%|██████████| 1/1 [12:43<00:00, 763.59s/it]
